In [1]:
import pandas as pd
import holidays
import datetime

In [2]:
df_z = pd.read_excel('2.xlsx', sheet_name='Заявки')
df_h = pd.read_excel('2.xlsx', sheet_name='Холды')

In [3]:
df_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10689 entries, 0 to 10688
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_zayavki  10689 non-null  object        
 1   hold        10689 non-null  object        
 2   hold_start  10689 non-null  datetime64[ns]
 3   hold_end    9582 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 334.2+ KB


In [4]:
df_z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42209 entries, 0 to 42208
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ИД заявки                  42209 non-null  object
 1   Вид оценки                 42209 non-null  object
 2   Сегмент                    42209 non-null  object
 3   подразделение эксперта     42209 non-null  object
 4   Цель рассмотрения заявки   42209 non-null  object
 5   Дата закрытия заявки       42209 non-null  object
 6   Дата открытия заявки       42209 non-null  object
 7   Назначение заявки          42209 non-null  object
 8   Автоматизация заявки       42209 non-null  object
 9   Классификатор Плышевского  42209 non-null  object
dtypes: object(10)
memory usage: 3.2+ MB


In [5]:
df_z[df_z['Дата открытия заявки'] == '-1'].count()

ИД заявки                    1885
Вид оценки                   1885
Сегмент                      1885
подразделение эксперта       1885
Цель рассмотрения заявки     1885
Дата закрытия заявки         1885
Дата открытия заявки         1885
Назначение заявки            1885
Автоматизация заявки         1885
Классификатор Плышевского    1885
dtype: int64

In [6]:
df_z = df_z[df_z['Дата открытия заявки'] !='-1']
df_z['Дата открытия заявки'] = df_z['Дата открытия заявки'].astype('datetime64[ns]')
df_z['Дата закрытия заявки'] = df_z['Дата закрытия заявки'].astype('datetime64[ns]')

В данном случае я заметил,что в некоторых строках в дате открытия стоит -1. Я бы уточнил что это значит и нет ли ошибки,
но так как сейчас такой возможности нет, я исключу эти заявки, темболее что их около 5% от общей массы.

В дальнейшем я буду учитывать -1 как отсутствие данных. Хоть его присутсиве в других столбцах и говорит что это не совсем верно.

Теперь даты это даты,а с доп параметрами можно будет разбираться дальше. По ходу задач.

In [7]:
merged_df = df_h.merge(df_z,left_on='id_zayavki', right_on='ИД заявки', how='left')
df_h['hold_end'] = merged_df['Дата закрытия заявки']
df_h['hold_end'] = df_h['hold_end'].astype('datetime64[ns]')

Я заполнил пропуски в окончании холдов с помощью дат закрытия заявкок

Все закрытия холда где null я сразу заполнил значениями

##### Определить общее время выполнения для каждой заявки как разницу между датами закрытия и открытия заявок (в сутках).

In [8]:
def date_difference(row):
    return row['Дата закрытия заявки'] - row['Дата открытия заявки']

df_z['Длительность'] = df_z.apply(date_difference, axis=1).dt.days

Я сделал это через функцию,которая находит разницу между концом и началом и выделяет из этого дни

##### Определить кол-во рабочих часов, необходимых для выполнения каждой заявки. 

In [ ]:
# Задаем рабочие часы и исключения
WORK_HOURS = {
    'monday': [(9, 0, 18, 0)],
    'tuesday': [(9, 0, 18, 0)],
    'wednesday': [(9, 0, 18, 0)],
    'thursday': [(9, 0, 18, 0)],
    'friday': [(9, 0, 16, 45)]
}

LUNCH_BREAK = (12, 15, 13, 0)
RU_HOLIDAYS = holidays.RU(years=[2022, 2023, 2024, 2025])  # Обновите года в зависимости от нужного периода

def is_working_time(dt):
    if dt.weekday() > 4 or dt.date() in RU_HOLIDAYS:
        return False
    day_name = dt.strftime('%A').lower()
    for start_h, start_m, end_h, end_m in WORK_HOURS.get(day_name, []):
        start_time = datetime.datetime(dt.year, dt.month, dt.day, start_h, start_m)
        end_time = datetime.datetime(dt.year, dt.month, dt.day, end_h, end_m)
        if start_time <= dt < end_time:
            return not (datetime.datetime(dt.year, dt.month, dt.day, *LUNCH_BREAK[:2]) <= dt < datetime.datetime(dt.year, dt.month, dt.day, *LUNCH_BREAK[2:]))
    return False

def calculate_working_seconds(start_dt, end_dt):
    current_dt = start_dt
    working_seconds = 0
    while current_dt < end_dt:
        if is_working_time(current_dt):
            working_seconds += 60  # Прибавляем минуту, если это рабочее время
        current_dt += datetime.timedelta(minutes=1)
    return working_seconds


# Вычисляем рабочие часы для каждой строки в DataFrame
df_z['working_seconds'] = df_z.apply(lambda row: calculate_working_seconds(row['Дата открытия заявки'], row['Дата закрытия заявки']), axis=1)
df_z